This notebook infers using models trained in [[Ubiquant] Google Colaboratory Training Code](https://www.kaggle.com/columbia2131/ubiquant-google-colaboratory-training-code).

In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
from glob import glob

import lightgbm as lgbm

def pred_lightgbm(X, dataset_dir, add_suffix=''):
    models = glob(os.path.join(dataset_dir, f'model/lgbm*{add_suffix}.pkl'))
    models = [pickle.load(open(model, 'rb')) for model in models]
    preds = np.array([model.predict(X) for model in models])
    preds = np.mean(preds, axis=0)
    return preds

In [ ]:
# =========================
# Inference
# =========================
feature_cols = [f'f_{i}' for i in range(300)]

import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()

for (test, sub) in iter_test:
    test_X = test[['investment_id'] + feature_cols]
    sub_base_pred = pred_lightgbm(test_X, '../input/ump-exp001-colabtraining', add_suffix='_base')
    sub['target'] = sub_base_pred
    env.predict(sub)